In [ ]:
pip install tabulate


In [1]:
#@markdown #1. Install the dependencies 
#dependency conflict 
#DEFINING THE FUNCTIONS

from tabulate import tabulate
from astropy.visualization import simple_norm

from ipywidgets import interact

import numpy as np
import skimage.io
import skimage.filters
import matplotlib.pyplot as plt
import os, random
import shutil 
import zipfile
from tifffile import imread, imsave


import pandas as pd


#Structural similarity (SSIM) index for measuring image quality #can probably change the sigma
def ssim(img1, img2):
  return structural_similarity(img1,img2,data_range=1.,full=True, gaussian_weights=True, use_sample_covariance=False, sigma=1.5)


def normalize(x, pmin=3, pmax=99.8, axis=None, clip=False, eps=1e-20, dtype=np.float32):
    """This function is adapted from Martin Weigert"""
    """Percentile-based image normalization."""

    mi = np.percentile(x,pmin,axis=axis,keepdims=True)
    ma = np.percentile(x,pmax,axis=axis,keepdims=True)
    return normalize_mi_ma(x, mi, ma, clip=clip, eps=eps, dtype=dtype)


def normalize_mi_ma(x, mi, ma, clip=False, eps=1e-20, dtype=np.float32):#dtype=np.float32
    """This function is adapted from Martin Weigert"""
    if dtype is not None:
        x   = x.astype(dtype,copy=False)
        mi  = dtype(mi) if np.isscalar(mi) else mi.astype(dtype,copy=False)
        ma  = dtype(ma) if np.isscalar(ma) else ma.astype(dtype,copy=False)
        eps = dtype(eps)

    try:
        import numexpr
        x = numexpr.evaluate("(x - mi) / ( ma - mi + eps )")
    except ImportError:
        x =                   (x - mi) / ( ma - mi + eps )

    if clip:
        x = np.clip(x,0,1)

    return x

def norm_minmse(gt, x, normalize_gt=True):
    """This function is adapted from Martin Weigert"""

    """
    normalizes and affinely scales an image pair such that the MSE is minimized  
     
    Parameters
    ----------
    gt: ndarray
        the ground truth image      
    x: ndarray
        the image that will be affinely scaled 
    normalize_gt: bool
        set to True of gt image should be normalized (default)
    Returns
    -------
    gt_scaled, x_scaled 
    """
    if normalize_gt:
        gt = normalize(gt, 0.1, 99.9, clip=False).astype(np.float32, copy = False)
    x = x.astype(np.float32, copy=False) - np.mean(x)    
    gt = gt.astype(np.float32, copy=False) - np.mean(gt)    
    scale = np.cov(x.flatten(), gt.flatten())[0, 1] / np.var(x.flatten())
    return gt, scale * x

In [4]:
#@markdown #5. Choose the folders that contain the data to analyse and run to load the data. 
#The source folder must contain both the unprocessed and processed file, or run it twice on both folders
#Solution here running on the raw data
#separate output folder

Source_folder = "/Users/secchim/Downloads/CellProfiler/movie_processing/2_channel_split_tif" #@param{type:"string"}
# Source_folder = "/Users/secchim/Downloads/CellProfiler/movies2_apply" #@param{type:"string"}
Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_processing/drift_assessment" #@param{type:"string"}

Analysis_type = "Z-slice" #@param ["Max_projection", "Z-slice"]

Reference_Frame = "Previous" #@param ["First", "Previous"]


#@markdown ##If not Max_projection, choose the Z plane to analyse
Z_plane =  5#@param {type:"number"}

# -------------------------------- Load the stack --------------------------------

# random_choice=random.choice(os.listdir(Source_folder))
for i in os.listdir(Source_folder):
  if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
    stack = imread(Source_folder+"/"+i)

    print(stack.shape)

    if Reference_Frame == "Previous":           
      print('The Previous frame will be used as a reference')

    if Reference_Frame == "First": 
      print('The First frame will be used as a reference')


    # perform the max projection

    if Analysis_type == "Max_projection":
      #make max projection
      maxproj = np.max(stack[:,:,:,:],axis = 1)
      print('---------------------------')
      print('max projection shape', maxproj.shape)


    if Analysis_type == "Z-slice":
      maxproj = stack[:,Z_plane,:,:]

      print('---------------------------')
      print('Stack shape:', maxproj.shape)



    # #Display one image

    # f=plt.figure(figsize=(16,8))
    # plt.subplot(1,2,1)
    # plt.imshow(maxproj[0], norm=simple_norm(maxproj[0], percent = 99), interpolation='nearest')

    # plt.axis('off')
    # plt.title('First frame');
    # plt.subplot(1,2,2)
    # plt.imshow(maxproj[1], norm=simple_norm(maxproj[1], percent = 99), interpolation='nearest')
    # plt.axis('off')
    # plt.title('Second frame');

(2, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (2, 512, 512)
(2, 12, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (2, 512, 512)
(6, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (6, 512, 512)
(5, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (5, 512, 512)
(22, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 512, 512)
(3, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (3, 512, 512)
(3, 12, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (3, 512, 512)
(7, 10, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (7, 512, 512)
(16, 7, 512, 512)
The Previous frame w

In [5]:
#@markdown #4. Process the raw data

import csv
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr

if Analysis_type == "Max_projection":
  Z_plane = ""

for i in os.listdir(Source_folder):
  if not os.path.isdir(os.path.join(Source_folder,i)):
    #if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'): # removed xyz_corrected in the scenario where the raw and corrected files are in different folders
    if i.endswith('Ch2.tif'):
      print('Running QC on: '+i)
      if i.startswith('VWF') and i.endswith('.tif'): #added that line because one hidden file was not tif in the folder
        stack = imread(Source_folder+"/"+i)

        if Analysis_type == "Max_projection":
      
          maxproj = np.max(stack[:,:,:,:],axis = 1)

        if Analysis_type == "Z-slice":
          maxproj = stack[:,Z_plane,:,:]

    # Open and create the csv file that will contain all the QC metrics
        with open(Result_folder+"/"+"QC_metrics_"+i+"_"+Analysis_type+str(Z_plane)+"_"+Reference_Frame+".csv", "w", newline='') as file:
            writer = csv.writer(file)

        # Write the header in the csv file
            writer.writerow(["image #","Z plane","Z plane + 1", "mSSIM", "NRMSE", "PSNR", "Pearson coefficient"])  

            # Initialize the lists
            Z_plane_list = []
            ssim_score_list = []
            Pearson_correlation_coefficient_list = []
              
        # Let's loop through the provided dataset in the QC folders

            for z in range(maxproj.shape[0]-1):

              Z_plane_list.append(z)            
          # -------------------------------- Load the data --------------------------------
              if Reference_Frame == "Previous":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[z]

              if Reference_Frame == "First":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[0]

          # Normalize the images wrt each other by minimizing the MSE between GT and Source image
              test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

          # -------------------------------- Calculate the metric maps and save them --------------------------------

          # Calculate the SSIM maps #structural similarity index
              index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

              ssim_score_list.append(index_SSIM_GTvsSource)

          #Save ssim_maps

                #img_SSIM_GTvsSource_8bit = (img_SSIM_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/SSIM_GTvsSource_"+shortname_no_PNG+'.tif',img_SSIM_GTvsSource_8bit)
          
          # Calculate the Root Squared Error (RSE) maps
              img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

          # Save SE maps
                #img_RSE_GTvsSource_8bit = (img_RSE_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/RSE_GTvsSource_"+shortname_no_PNG+'.tif',img_RSE_GTvsSource_8bit)


          # -------------------------------- Calculate the RSE metrics and save them --------------------------------

          # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
              NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
            
          # We can also measure the peak signal to noise ratio between the images
              PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)


              cm1 = np.corrcoef(test_GT_norm.flat, test_source_norm.flat) #outputs a flat number
              r1 = cm1[0, 1]
              Pearson_correlation_coefficient_list.append(r1)

              writer.writerow([i,str(z),str(z+1),str(index_SSIM_GTvsSource),str(NRMSE_GTvsSource),str(PSNR_GTvsSource),str(r1)])

  # All data is now processed saved




Running QC on: VWF_060_MS220408_M1_MOVIE4_P4_2_Ch2.tif
Running QC on: VWF_048_MS220119_M1_homeostasis_movie_2_P17DP_Ch2.tif
Running QC on: VWF_060_MS220408_M1_MOVIE3_P11DP_Ch2.tif
Running QC on: VWF_060_MS220408_M1_MOVIE2_P4GREENVESSEL_Ch2.tif
Running QC on: VWF_066_MS220522_M1_movie2_P2kgreen_Ch2.tif
Running QC on: VWF_048_MS220120_M2_movie14_depletion_P9_Ch2.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie10depletion_P12DP_Ch2.tif
Running QC on: VWF_063_MS220428_m1_movie2_P4_Ch2.tif
Running QC on: VWF_043_MS211125_m3-homeostasis_movie5_1.5x_P23+_Ch2.tif
Running QC on: VWF_043_MS211126_m4-homeostasismovie_2_P1<3_Ch2.tif
Running QC on: VWF_048_MS220120_M2_movie19_depletion_P14_Ch2.tif
Running QC on: VWF_048_MS220120_M2_movie16_depletion_P11_Ch2.tif
Running QC on: VWF_048_MS220120_M2_movie11_depletion_P9_Ch2.tif
Running QC on: VWF_045_MS211208_M2_movie8_P27_Ch2.tif
Running QC on: VWF_066_MS220522_M4_pmovie2_P2kuomk_Ch2.tif
Running QC on: VWF_043_MS211125_m3-homeostasis_movie2_M1_Ch2.ti

In [2]:
#@markdown #3. Choose the folders that contain the data to analyse and run to load the data. 
#The source folder must contain both the unprocessed and processed file, or run it twice on both folders
#Here running on the processed files
#separate output folder

# Source_folder = "/Users/secchim/Downloads/CellProfiler/movie_processing/4_drift_corrected" #@param{type:"string"}
Source_folder = "/Volumes/LUIS1_MS/Raw_Data/imaging/movie_processing/4_drift_corrected" #@param{type:"string"}
# Source_folder = "/Users/secchim/Downloads/CellProfiler/movies2_apply" #@param{type:"string"}
Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_processing/drift_assessment" #@param{type:"string"}

Analysis_type = "Z-slice" #@param ["Max_projection", "Z-slice"]

Reference_Frame = "Previous" #@param ["First", "Previous"]


#@markdown ##If not Max_projection, choose the Z plane to analyse
Z_plane =  5#@param {type:"number"}

# -------------------------------- Load the stack --------------------------------

# random_choice=random.choice(os.listdir(Source_folder))
for i in os.listdir(Source_folder):
  if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
    stack = imread(Source_folder+"/"+i)

    print(stack.shape)

    if Reference_Frame == "Previous":           
      print('The Previous frame will be used as a reference')

    if Reference_Frame == "First": 
      print('The First frame will be used as a reference')


    # perform the max projection

    if Analysis_type == "Max_projection":
      #make max projection
      maxproj = np.max(stack[:,:,:,:],axis = 1)
      print('---------------------------')
      print('max projection shape', maxproj.shape)


    if Analysis_type == "Z-slice":
      maxproj = stack[:,Z_plane,:,:]

      print('---------------------------')
      print('Stack shape:', maxproj.shape)



    # #Display one image

    # f=plt.figure(figsize=(16,8))
    # plt.subplot(1,2,1)
    # plt.imshow(maxproj[0], norm=simple_norm(maxproj[0], percent = 99), interpolation='nearest')

    # plt.axis('off')
    # plt.title('First frame');
    # plt.subplot(1,2,2)
    # plt.imshow(maxproj[1], norm=simple_norm(maxproj[1], percent = 99), interpolation='nearest')
    # plt.axis('off')
    # plt.title('Second frame');



(9, 20, 1427, 1061)
The Previous frame will be used as a reference
---------------------------
Stack shape: (9, 1427, 1061)
(5, 12, 510, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (5, 510, 510)
(21, 11, 506, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 506, 510)
(6, 12, 511, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (6, 511, 512)
(18, 18, 498, 508)
The Previous frame will be used as a reference
---------------------------
Stack shape: (18, 498, 508)
(5, 12, 422, 487)
The Previous frame will be used as a reference
---------------------------
Stack shape: (5, 422, 487)
(5, 12, 511, 511)
The Previous frame will be used as a reference
---------------------------
Stack shape: (5, 511, 511)
(11, 20, 502, 499)
The Previous frame will be used as a reference
---------------------------
Stack shape: (11, 502, 499)
(2, 14, 512, 511)
The Previous

TiffFileError: not a TIFF file b'\x00\x05\x16\x07'

Running QC on: VWF_048_MS220118_M3_movie2homeostasis_P6DP+_Ch2.tif


(22, 14, 509, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 509, 510)
(21, 12, 512, 512)
The Previous frame will be used as a reference
---------------------------
Stack shape: (21, 512, 512)
(22, 14, 509, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 509, 510)
(22, 14, 509, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 509, 510)
(22, 14, 509, 510)
The Previous frame will be used as a reference
---------------------------
Stack shape: (22, 509, 510)


In [3]:
#@markdown #6. Process the corrected data

import csv
from skimage.metrics import structural_similarity
from skimage.metrics import peak_signal_noise_ratio as psnr

if Analysis_type == "Max_projection":
  Z_plane = ""

for i in os.listdir(Source_folder):
  if not os.path.isdir(os.path.join(Source_folder,i)):
    if i.endswith('Ch2.tif') or i.endswith('xyzCorrected.tif'):
      print('Running QC on: '+i)
      if i.startswith('VWF') and i.endswith('.tif'): #added that line because one hidden file was not tif in the folder
        stack = imread(Source_folder+"/"+i)

        if Analysis_type == "Max_projection":
      
          maxproj = np.max(stack[:,:,:,:],axis = 1)

        if Analysis_type == "Z-slice":
          maxproj = stack[:,Z_plane,:,:]

    # Open and create the csv file that will contain all the QC metrics
        with open(Result_folder+"/"+"QC_metrics_"+i+"_"+Analysis_type+str(Z_plane)+"_"+Reference_Frame+".csv", "w", newline='') as file:
            writer = csv.writer(file)

        # Write the header in the csv file
            writer.writerow(["image #","Z plane","Z plane + 1", "mSSIM", "NRMSE", "PSNR", "Pearson coefficient"])  

            # Initialize the lists
            Z_plane_list = []
            ssim_score_list = []
            Pearson_correlation_coefficient_list = []
              
        # Let's loop through the provided dataset in the QC folders

            for z in range(maxproj.shape[0]-1):

              Z_plane_list.append(z)            
          # -------------------------------- Load the data --------------------------------
              if Reference_Frame == "Previous":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[z]

              if Reference_Frame == "First":           
                test_GT = maxproj[z+1]                     
                test_source = maxproj[0]

          # Normalize the images wrt each other by minimizing the MSE between GT and Source image
              test_GT_norm,test_source_norm = norm_minmse(test_GT, test_source, normalize_gt=True)

          # -------------------------------- Calculate the metric maps and save them --------------------------------

          # Calculate the SSIM maps #structural similarity index
              index_SSIM_GTvsSource, img_SSIM_GTvsSource = ssim(test_GT_norm, test_source_norm)

              ssim_score_list.append(index_SSIM_GTvsSource)

          #Save ssim_maps

                #img_SSIM_GTvsSource_8bit = (img_SSIM_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/SSIM_GTvsSource_"+shortname_no_PNG+'.tif',img_SSIM_GTvsSource_8bit)
          
          # Calculate the Root Squared Error (RSE) maps
              img_RSE_GTvsSource = np.sqrt(np.square(test_GT_norm - test_source_norm))

          # Save SE maps
                #img_RSE_GTvsSource_8bit = (img_RSE_GTvsSource* 255).astype("uint8")
                #io.imsave(QC_model_path+'/'+QC_model_name+"/Quality Control/"+str(checkpoints)+"/RSE_GTvsSource_"+shortname_no_PNG+'.tif',img_RSE_GTvsSource_8bit)


          # -------------------------------- Calculate the RSE metrics and save them --------------------------------

          # Normalised Root Mean Squared Error (here it's valid to take the mean of the image)
              NRMSE_GTvsSource = np.sqrt(np.mean(img_RSE_GTvsSource))
            
          # We can also measure the peak signal to noise ratio between the images
              PSNR_GTvsSource = psnr(test_GT_norm,test_source_norm,data_range=1.0)


              cm1 = np.corrcoef(test_GT_norm.flat, test_source_norm.flat) #outputs a flat number
              r1 = cm1[0, 1]
              Pearson_correlation_coefficient_list.append(r1)

              writer.writerow([i,str(z),str(z+1),str(index_SSIM_GTvsSource),str(NRMSE_GTvsSource),str(PSNR_GTvsSource),str(r1)])

  # All data is now processed saved

Running QC on: VWF_045_MS211208_M2_movie7_P26_Ch4_xyzCorrected.tif


/opt/anaconda3/envs/New_environment220920/lib/python3.8/site-packages/skimage/metrics/simple_metrics.py:163: RuntimeWarning: divide by zero encountered in double_scalars
  return 10 * np.log10((data_range ** 2) / err)
/opt/anaconda3/envs/New_environment220920/lib/python3.8/site-packages/numpy/lib/function_base.py:2853: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/var/folders/m1/973p2py16d5bwyw1r5whmn0hh54wfr/T/ipykernel_24407/2526091436.py:78: RuntimeWarning: invalid value encountered in double_scalars
  scale = np.cov(x.flatten(), gt.flatten())[0, 1] / np.var(x.flatten())


Running QC on: VWF_048_MS220120_M2_movie16_depletion_P11_Ch2_xyzCorrected.tif
Running QC on: VWF_043_MS211118_m1-p5_11_12_pltdepletionmovie_P12_Ch4_xyzCorrected.tif
Running QC on: VWF_063_MS220428_m1_movie3_P9dimdp_Ch4_xyzCorrected.tif
Running QC on: VWF_043_MS211126_m4-pltdepletionmovie3_P17+_Ch2_xyzCorrected.tif
Running QC on: VWF_060_MS220408_M1_MOVIE2_P10_Ch4_xyzCorrected.tif
Running QC on: VWF_048_MS220120_M2_movie16_depletion_P9_Ch3_xyzCorrected.tif
Running QC on: VWF_043_MS211126_m4-pltdepletionmovie2_P7++_Ch3_xyzCorrected.tif
Running QC on: VWF_049_MS220201_M1_Snap_movie6depletion_P10spot_Ch4_xyzCorrected.tif
Running QC on: VWF_043_MS211118_m1-p5_11_12_homeostasismovie_P5_Ch2_xyzCorrected.tif
Running QC on: VWF_063_MS220428_m1_movie2_P9dimdp_Ch3_xyzCorrected.tif
Running QC on: VWF_043_MS211118_m1-p5_11_12_homeostasismovie_P12_Ch3_xyzCorrected.tif
Running QC on: VWF_053_MS220209_M1_movie8depletion_P12kuo_Ch2_xyzCorrected.tif
Running QC on: VWF_048_MS220119_M1_pltdepletion_movie_

In [11]:
#@markdown #. Showing and SAVING all 
##MATCHING UNCORRECTED AND FAST4DREG CORRECTED FILES

Result_folder = "/Users/secchim/Downloads/CellProfiler/movie_processing/drift_assessment" #@param{type:"string"}

QCResult_folder='/Users/secchim/Downloads/CellProfiler/movie_processing/drift_assessment_png'

def show_QC_results(file1): #file1, file2, requires a list as an argument
  # files= os.listdir(Result_folder)#
  # file1=[i for i in os.listdir(Result_folder) if "Corrected" not in i] #if file already has corrected don't look at them , runs quicker than forloop, called list comprehension
  df1 = pd.read_csv (Result_folder+"/"+file1)
  ind=file1.index(".tif")
  rep=file1[ind:]
  file2=file1.replace(rep, '_xyzCorrected'+rep)
  # file3=file1.replace(rep, 'drift'+rep)
  df2 = pd.read_csv (Result_folder+"/"+file2)
  # df3 = pd.read_csv (Result_folder+"/"+file3)
  df1.set_index("image #", inplace=True)
  df2.set_index("image #", inplace=True)
  # df3.set_index("image #", inplace=True)
  # print(tabulate((df1,df2), headers='keys', tablefmt='psql'))

  Z_plane_list1 = df1['Z plane + 1'].values.tolist()
  Z_plane_list2 = df2['Z plane + 1'].values.tolist()
  # Z_plane_list3 = df3['Z plane + 1'].values.tolist()
  ssim_score_list1 = df1['mSSIM'].values.tolist()
  ssim_score_list2 = df2['mSSIM'].values.tolist()
  # ssim_score_list3 = df3['mSSIM'].values.tolist()
  Pearson_correlation_coefficient_list1 = df1['Pearson coefficient'].values.tolist()
  Pearson_correlation_coefficient_list2 = df2['Pearson coefficient'].values.tolist()
  # Pearson_correlation_coefficient_list3 = df3['Pearson coefficient'].values.tolist()



# -------------------------------- Display --------------------------------

  plt.figure(figsize=(20,5))
  plt.plot(Z_plane_list1, ssim_score_list1, label="SSIM_raw")
  plt.plot(Z_plane_list2, ssim_score_list2, label="SSIM_Fast4Dcorrected")
  # plt.plot(Z_plane_list3, ssim_score_list3, label="SSIM_ImageJcorrected")
  plt.title('Timepoints vs. SSIM')
  plt.ylabel('SSIM')
  plt.xlabel('Timepoints')
  plt.legend()
  # plt.savefig(full_QC_model_path+'/Quality Control/SSIMvsCheckpoint_data.png',bbox_inches='tight',pad_inches=0)
  plt.savefig(QCResult_folder+("/"+file1+'SSIMvsCheckpoint_data.png'),bbox_inches='tight',pad_inches=0)
  #plt.show()
  plt.close()

  plt.figure(figsize=(20,5))
  plt.plot(Z_plane_list1, Pearson_correlation_coefficient_list1, label="Pearson coefficient_raw")
  plt.plot(Z_plane_list2, Pearson_correlation_coefficient_list2, label="Pearson coefficient_Fast4Dcorrected")
  # plt.plot(Z_plane_list3, Pearson_correlation_coefficient_list3, label="Pearson coefficient_ImageJcorrected")
  plt.title('Timepoints vs. Pearson coefficient')
  plt.ylabel('Pearson coefficient')
  plt.xlabel('Timepoints')
  plt.legend()
#plt.savefig(full_QC_model_path+'/Quality Control/lpipsvsCheckpoint_data.png',bbox_inches='tight',pad_inches=0)
  plt.savefig(QCResult_folder+("/"+file1+'lpipsvsCheckpoint_data.png'),bbox_inches='tight',pad_inches=0)
 # plt.show()
  plt.close()


error_list=[]
for i in os.listdir(Result_folder):
  if (".tif" in i and "Corrected" not in i and ".DS_Store" not in i): 
    # print(i)
    try:
      show_QC_results(i)
      print(i)
    except:
      print("error with "+i)
      error_list.append(i)


#vasculature P12 is the problem
    

QC_metrics_VWF_045_MS211208_M2_movie7_P25_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie8depletion_P16DP_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_048_MS220120_M2_movie16_depletion_P14_Ch2.tif_Z-slice5_Previous.csv
error with QC_metrics_VWF_066_MS220522_M4_pmovie1_P5bigDP_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_062_MS220421_M5_MOVIE3_P16_2KUO_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie8depletion_P14partlygreen_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_053_MS220209_M1_movie3depletion_P26kuovascu_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_045_MS211208_M2_movie3_P21_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_060_MS220408_M1_MOVIE3_P8_Ch2.tif_Z-slice5_Previous.csv
QC_metrics_VWF_063_MS220428_m1_movie2_P15_Ch2.tif_Z-slice5_Previous.csv
error with QC_metrics_VWF_048_MS220120_M2_movie18_depletion_P11_Ch2.tif_Z-slice5_Previous.csv
error with QC_metrics_VWF_043_MS211118_m1-p5_11_12_homeostasismovie_vasculature_P11_Ch2.tif_Z-slice5_Previo

In [13]:
print(len(error_list))
print(error_list)

73
['QC_metrics_VWF_066_MS220522_M4_pmovie1_P5bigDP_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220120_M2_movie18_depletion_P11_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_043_MS211118_m1-p5_11_12_homeostasismovie_vasculature_P11_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_053_MS220209_M1_movie5depletion_P16DP_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220119_M1_homeostasis_movie_3_P16DP_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220120_M2_movie11_depletion_P9_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220118_M3_movie2homeostasis_P13+_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_060_MS220408_M1_MOVIE3_P1DP_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_053_MS220209_M1_movie8depletion_P17kuo_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_060_MS220408_M1_MOVIE3_P11DP_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220120_M2_movie13_depletion_P14_Ch2.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_048_MS220120_M2_movie13_depletion_

In [11]:
Result_folder = "/Users/secchim/Downloads/CellProfiler/QC2" #@param{type:"string"}
QCResult_folder='/Users/secchim/Downloads/CellProfiler/QC2png'

print(os.listdir(Result_folder))


['QC_metrics_VWF_045_MS211208_M2_movie7_P27_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P25_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie6_P23_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P27_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie3_P21_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch4_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie7_P25_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie6_P23_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie3_P21_Ch2_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2_movie8_P26_Ch1_xyzCorrected.tif_Z-slice5_Previous.csv', 'QC_metrics_VWF_045_MS211208_M2

In [8]:
#This cell will return a list of movies that should be checked because pearson correlation is low


for file1 in os.listdir(Result_folder):
  if ".tif" in file1 and "Corrected" not in file1: 
    ind=file1.index(".tif")
    rep=file1[ind:]
    file2=file1.replace(rep, '_xyzCorrected'+rep)
    df2 = pd.read_csv (Result_folder+"/"+file2)

    Pearson_correlation_coefficient_list2 = df2['Pearson coefficient'].values.tolist()
        
    for t in Pearson_correlation_coefficient_list2 :
        if t<0.4:
            print(file2)
            break

In [ ]:
#This cell will move the movies that were better uncorrected into a new folder, move the movies that were better corrected into a new folder
#then add xyzCorrected to the name of the movies that were better uncorrected => to allow downstream processing

#Here Result_folder is the QC/ drift assessment folder #same as above
#Here Split_folder is the split channel folder

Split_folder="/Users/secchim/Downloads/CellProfiler/movie_processing/2_channel_split_tif"

for file1 in os.listdir(Result_folder):
  if ".tif" in file1 and "Corrected" not in file1: 
    ind=file1.index(".tif")
    rep=file1[ind:]
    file2=file1.replace(rep, '_xyzCorrected'+rep)
    df1 = pd.read_csv (Result_folder+"/"+file1)
    df2 = pd.read_csv (Result_folder+"/"+file2)

    Pearson_correlation_coefficient_list2 = df2['Pearson coefficient'].values.tolist()
    differential_pearson=sum(df2["Pearson coefficient"]-df1["Pearson coefficient"])/len(df2["Pearson coefficient"])
    if differential_pearson<0.05:
      print("moving "+file1)
      os.rename(Split_folder+"/"+file1, Split_folder+"/BetterUncorrected/"+file2)
      
        
    for t in Pearson_correlation_coefficient_list2 :
        if t<0.4:
            print(file2)
            os.rename(Split_folder+"/"+file1, Split_folder+"/BetterUncorrected/"+file2)
            break